In [1]:
import tensorflow as tf
import os
from constants import UNKNOWN_FACES_PATH, MODEL_INPUT_IMAGE_DIMENSIONS, PATH_TO_FAILED_MODEL_EVALUATION_IMAGES, MODEL_0_FAILED_PATH, MODEL_1_FAILED_PATH, MODEL_2_FAILED_PATH, KNOWN_FACE_1_PATH, KNOWN_FACE_2_PATH, MODEL_PATH
from helpers import preprocess, deprocess_image, display_progressbar
import numpy as np
import time
from face_tracking import face_detection
from PIL import Image
import shutil
import math

model = tf.keras.models.load_model(os.path.join(MODEL_PATH, 'model_final_with_josh_1.h5'))

#### Check model manually

In [4]:
face_detection(model=model, required_bound=.8, certainty_bound=.1)

#### Delete old files

In [3]:
for directory in os.listdir(PATH_TO_FAILED_MODEL_EVALUATION_IMAGES):
    if os.path.isdir(os.path.join(PATH_TO_FAILED_MODEL_EVALUATION_IMAGES, directory)):
        for file in os.listdir(os.path.join(PATH_TO_FAILED_MODEL_EVALUATION_IMAGES, directory)):
            file_path = os.path.join(PATH_TO_FAILED_MODEL_EVALUATION_IMAGES, directory, file)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as e:
                print('Failed to delete %s. Reason: %s' % (file_path, e))

### Evaluate model on all data

#### Evaluate on photos of Stephen

In [ ]:
num_failures_0 = 0
t_orig = time.time()
total_photos = len(os.listdir(KNOWN_FACE_1_PATH))
total_processed = 0
t = time.time()
for face in os.listdir(KNOWN_FACE_1_PATH):
    face_path = os.path.join(KNOWN_FACE_1_PATH, face)
    im = deprocess_image(preprocess(face_path, MODEL_INPUT_IMAGE_DIMENSIONS))
    pred = model.predict(np.array([im]), verbose=0)[0]
    if not math.isclose(max(pred), pred[0]):
        num_failures_0 += 1
        print("\rNumber of failures for stephen images: " + str(num_failures_0) + "; " + str(pred))
        im = Image.fromarray(im)
        im.save(os.path.join(MODEL_0_FAILED_PATH, face))
    total_processed += 1
    t = display_progressbar(t1=t, t_orig=t_orig, total_photos=total_photos, total_processed=total_processed)

#### Evaluate on photos of Josh

In [ ]:
num_failures_1 = 0
t_orig = time.time()
total_photos = len(os.listdir(KNOWN_FACE_2_PATH))
total_processed = 0
t = time.time()
for face in os.listdir(KNOWN_FACE_2_PATH):
    face_path = os.path.join(KNOWN_FACE_2_PATH, face)
    im = deprocess_image(preprocess(face_path, MODEL_INPUT_IMAGE_DIMENSIONS))
    pred = model.predict(np.array([im]), verbose=0)[0]
    if not math.isclose(max(pred), pred[1]):
        num_failures_1 += 1
        print("\rNumber of failures for josh images: " + str(num_failures_1) + "; " + str(pred))
        im = Image.fromarray(im)
        im.save(os.path.join(MODEL_1_FAILED_PATH, face))
    total_processed += 1
    t = display_progressbar(t1=t, t_orig=t_orig, total_photos=total_photos, total_processed=total_processed)

2024-03-12 07:37:22.382100: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


Number of failures for josh images: 1; [0.66399145 0.26118612 0.01771503]
Number of failures for josh images: 2; [0.4339334  0.35972443 0.60509497]
Number of failures for josh images: 3; [0.8562883  0.25785965 0.01228777]
Number of failures for josh images: 4; [0.0214674  0.41640815 0.9904144 ]
Number of failures for josh images: 5; [0.20888318 0.03774055 0.16610938]
Number of failures for josh images: 6; [0.83199763 0.722028   0.01326547]
Number of failures for josh images: 7; [0.7404043  0.70939624 0.00402061]
Number of failures for josh images: 8; [5.2527833e-01 7.3313173e-03 1.2812107e-05]
Number of failures for josh images: 9; [0.06356975 0.40823823 0.5064122 ]
Number of failures for josh images: 10; [0.78173953 0.12328588 0.04812   ]
Number of failures for josh images: 11; [0.01821495 0.38302797 0.39624777]
Number of failures for josh images: 12; [0.83350146 0.7680585  0.06849601]
Number of failures for josh images: 13; [0.8912652  0.76125294 0.00173752]
Number of failures for jo

#### Evaluate on photos of unknown people

In [ ]:
num_failures_2 = 0
t_orig = time.time()
total_photos = len(os.listdir(UNKNOWN_FACES_PATH))
total_processed = 0
t = time.time()
for face in os.listdir(UNKNOWN_FACES_PATH):
    face_path = os.path.join(UNKNOWN_FACES_PATH, face)
    im = deprocess_image(preprocess(face_path, MODEL_INPUT_IMAGE_DIMENSIONS))
    pred = model.predict(np.array([im]), verbose=0)[0]
    if not math.isclose(max(pred), pred[2]):
        num_failures_2 += 1
        print("\rNumber of failures for unknown images: " + str(num_failures_2) + "; " + str(pred))
        im = Image.fromarray(im)
        im.save(os.path.join(MODEL_2_FAILED_PATH, face))
    total_processed += 1
    t = display_progressbar(t1=t, t_orig=t_orig, total_photos=total_photos, total_processed=total_processed)

In [ ]:
print("Total failures: " + str(num_failures_0 + num_failures_1 + num_failures_2))